<a href="https://colab.research.google.com/github/dquerales/jupyter-automation-github-actions/blob/main/notebooks/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training

____

## Load libraries

In [ ]:
from pycaret.regression import *
import pandas as pd
import numpy as np

____

## Load data

In [ ]:
df = pd.read_csv('datasets/raw_data.csv')

In [ ]:
df.head()

,date,price
0,2022-05-07 00:00:00,36013.03
1,2022-05-08 00:00:00,35471.42
2,2022-05-09 00:00:00,34082.21
3,2022-05-10 00:00:00,30175.71
4,2022-05-11 00:00:00,31003.93


____

## Data Preparation


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['day'] = df["date"].dt.day
df['month'] = df["date"].dt.month
df['year'] = df["date"].dt.year

In [ ]:
df['series'] = np.arange(1,len(df)+1)

In [ ]:
df.head()

,date,price,day,month,year,series
0,2022-05-07,36013.03,7,5,2022,1
1,2022-05-08,35471.42,8,5,2022,2
2,2022-05-09,34082.21,9,5,2022,3
3,2022-05-10,30175.71,10,5,2022,4
4,2022-05-11,31003.93,11,5,2022,5


In [ ]:
import plotly.express as px
fig = px.line(df, x="date", y="price", template = 'plotly_dark')
fig.show()

In [ ]:
train_split = round(len(df)*0.7)
test_split = len(df) - train_split

In [ ]:
train = df.head(train_split)
test = df.tail(test_split)

____

## Modelling

In [ ]:
time_series = setup(data = train, 
                    test_data = test, 
                    target = 'price', 
                    fold_strategy = 'timeseries', 
                    # numeric_features = ['day', 'month', 'year', 'series'], ignore_features = ['date'], 
                    numeric_features = ['series'], ignore_features = ['day', 'month', 'year'], 
                    transform_target = True, 
                    session_id = 123)

,Description,Value
0,Session id,123
1,Target,price
2,Target type,Regression
3,Original data shape,"(366, 6)"
4,Transformed data shape,"(366, 5)"
5,Transformed train set shape,"(256, 5)"
6,Transformed test set shape,"(110, 5)"
7,Ignore features,3
8,Numeric features,1
9,Date features,1


### Compare models

In [ ]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1853.3572,8579176.6941,2247.0752,-6.3893,0.1034,0.0910,0.1760
et,Extra Trees Regressor,1981.3389,8801502.5024,2371.0812,-6.9149,0.1102,0.0999,0.3090
rf,Random Forest Regressor,1803.2192,7748188.7482,2157.0314,-7.6299,0.1008,0.0894,0.5770
dt,Decision Tree Regressor,1990.5171,10320342.8872,2427.7753,-8.1380,0.1103,0.0981,0.1290
ada,AdaBoost Regressor,1802.7050,7247413.4882,2123.3031,-8.2656,0.1000,0.0891,0.1520
omp,Orthogonal Matching Pursuit,2113.5953,10715097.4677,2434.0677,-9.4094,0.1178,0.0996,0.2150
lar,Least Angle Regression,2055.2620,10065716.3363,2373.8168,-9.5768,0.1161,0.0971,0.1580
ridge,Ridge Regression,2173.1562,10617629.3433,2477.2366,-9.8110,0.1230,0.1037,0.1140
knn,K Neighbors Regressor,1843.9281,7685197.5547,2200.5449,-12.2209,0.1043,0.0935,0.2130
lightgbm,Light Gradient Boosting Machine,2368.0159,11805418.2352,2764.5854,-16.4633,0.1267,0.1186,0.1370


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

### Tune model

In [ ]:
tuned_best_model = tune_model(best_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4724.2289,39965225.6643,6321.8056,-0.9356,0.2527,0.2223
1,1945.1995,5786520.5135,2405.5188,-8.7959,0.1167,0.0960
2,2130.3486,5389658.4378,2321.5638,-7.6127,0.1061,0.0920
3,1860.7973,5233550.9889,2287.6956,-0.8482,0.1040,0.0892
4,3033.9359,10129593.7383,3182.7023,-9.9530,0.1501,0.1545
5,3403.5338,11721471.2576,3423.6634,-84.2912,0.1631,0.1760
6,2273.2351,5740896.0003,2396.0167,-9.0138,0.1145,0.1149
7,5197.3949,27165170.5461,5212.0217,-177.4163,0.2740,0.3143
8,3578.7193,12906032.0456,3592.4966,-129.6279,0.1919,0.2107


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


### Performance

In [ ]:
evaluate_model(best_model)

### Finalize model

In [ ]:
final_best_model = finalize_model(best_model)

____

## Test data

In [ ]:
predictions = predict_model(best_model, data = test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,20485.6942,419744993.3636,20487.6791,-1052413016880658120704.0000,9.3207,24547.9040


In [ ]:
predictions

,date,day,month,year,series,price,prediction_label
256,2023-01-18,18,1,2023,257,21145.179688,20894.516749
257,2023-01-19,19,1,2023,258,20672.779297,20213.282615
258,2023-01-20,20,1,2023,259,21087.759766,20128.466998
259,2023-01-21,21,1,2023,260,22693.449219,20128.466998
260,2023-01-22,22,1,2023,261,22772.500000,20128.466998
...,...,...,...,...,...,...,...
361,2023-05-03,3,5,2023,362,28678.289062,20365.698497
362,2023-05-04,4,5,2023,363,29007.660156,20365.698497
363,2023-05-05,5,5,2023,364,28851.839844,20765.575337
364,2023-05-06,6,5,2023,365,29535.380859,20765.575337


In [ ]:
fig = px.line(predictions, x="date", y=["price", 'prediction_label'], template = 'plotly_dark')
fig.show()

In [ ]:
predictions.to_csv('datasets/test_dataset_labeled.csv', index=False)

____

### Save model

In [ ]:
save_model(final_best_model, 'models/best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('target_transformation',
                  TransformerWrapperWithInverse(transformer=TargetTransformer(estimator=PowerTransformer(standardize=False)))),
                 ('date_feature_extractor',
                  TransformerWrapper(include=['date'],
                                     transformer=ExtractDateTimeFeatures())),
                 ('numerical_imputer',
                  TransformerWrapper(include=['series'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', GradientBoostingRegressor(random_state=123))]),
 'model.pkl')